## Set up

In [1]:
! pip install pytorch-nlp

     |████████████████████████████████| 90 kB 6.0 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/dl_project/counterfactual_fairness_emotional_recognition-regularization.zip /content/
!unzip counterfactual_fairness_emotional_recognition-regularization.zip

Archive:  counterfactual_fairness_emotional_recognition-regularization.zip
bdbddcaf5be91b1547a8370c5c1c8af6c0f62085
   creating: counterfactual_fairness_emotional_recognition-regularization/
  inflating: counterfactual_fairness_emotional_recognition-regularization/.gitignore  
   creating: counterfactual_fairness_emotional_recognition-regularization/.vscode/
  inflating: counterfactual_fairness_emotional_recognition-regularization/.vscode/launch.json  
  inflating: counterfactual_fairness_emotional_recognition-regularization/.vscode/settings.json  
  inflating: counterfactual_fairness_emotional_recognition-regularization/README.md  
   creating: counterfactual_fairness_emotional_recognition-regularization/archive/
   creating: counterfactual_fairness_emotional_recognition-regularization/archive/baseline_ACRNN/
  inflating: counterfactual_fairness_emotional_recognition-regularization/archive/baseline_ACRNN/3-D.pdf  
  inflating: counterfactual_fairness_emotional_recognition-regularizati

In [17]:
!cp /content/drive/MyDrive/11785_Project/IEMOCAP_pkls.tar.gz /content/counterfactual_fairness_emotional_recognition-regularization/data

% cd /content/counterfactual_fairness_emotional_recognition-regularization/data
!tar -xvf IEMOCAP_pkls.tar.gz

/content/counterfactual_fairness_emotional_recognition-regularization/data
IEMOCAP_train.pkl
IEMOCAP_train_converted.pkl
IEMOCAP_eval.pkl
IEMOCAP_valid_data.npy
IEMOCAP_valid_gender.npy
IEMOCAP_valid_label.npy
IEMOCAP_test_data.npy
IEMOCAP_test_gender.npy
IEMOCAP_test_label.npy


# Train 10-20 models

In [1]:
# change the epoch num in train.py if necessary

% cd /content/counterfactual_fairness_emotional_recognition-regularization

for i in range(10):
    !python /content/counterfactual_fairness_emotional_recognition-regularization/train.py
    !cp -r -n /content/counterfactual_fairness_emotional_recognition-regularization/checkpoint /content/drive/MyDrive/dl_project/experiment_checkpoints
    ! cp -r -n /content/counterfactual_fairness_emotional_recognition-regularization/log /content/drive/MyDrive/dl_project/training_logs

Streaming output truncated to the last 5000 lines.
 [  0  14  16 184]]
Best Valid Confusion Matrix:["ang","sad","hap","neu"]
[[ 20   3   4   0]
 [  4  85  10   8]
 [ 23  12  22  23]
 [ 25 107  26  56]]
Learning Rate: 0.001
*****************************************************************
  0% 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1320: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silenc

## Get test result

In [2]:
% cd /content/counterfactual_fairness_emotional_recognition-regularization

import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import IEMOCAPEval
import os
from train import test

/content/counterfactual_fairness_emotional_recognition-regularization


In [22]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score as recall
from sklearn.metrics import confusion_matrix as confusion
from datasets import IEMOCAPTrain, IEMOCAPEval
from utils.training_tracker import TrainingTracker
from utils.fairness_eval import FairnessEvaluation


In [4]:
model_dir = "/content/drive/MyDrive/dl_project/experiment_checkpoints/ensembled_models"
saved_models = os.listdir(model_dir)
saved_models = [os.path.join(model_dir, i) for i in saved_models]

In [6]:
# ! pwd

/content/counterfactual_fairness_emotional_recognition-regularization


In [11]:
test_dataset = IEMOCAPEval(partition='test')
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=1, pin_memory=True)
criterion = nn.CrossEntropyLoss()

# (test_sample_size, categories, models)
y_pred_enmodel = np.zeros(shape=(428, 4, len(saved_models)),dtype=np.float32)

In [13]:
# test return y_pred_test

for i in range(len(saved_models)):
    print("Testing model"+str(i))
    CHECKPOINT_MODEL_DIR = saved_models[i]
    model = torch.load(CHECKPOINT_MODEL_DIR).cuda()
    y_pred = test(model, test_loader, test_dataset, criterion)
    y_pred_enmodel[:,:,i] = y_pred

Testing model0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1320: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


*****************************************************************
Evaluation on Test Set:
Test cost: 1.25
Test accuracy: 0.472
Test UAR: 0.532
Test Confusion Matrix:["ang","sad","hap","neu"]
[[22  1  3  3]
 [14 68  8 20]
 [18  5 25 23]
 [43 52 36 87]]
Fairness Scores (in terms of equal opportunities):
["ang","sad","hap","neu"]
[0.93333333 0.66576819 0.86666667 0.75497788]
*****************************************************************
Testing model1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1320: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


*****************************************************************
Evaluation on Test Set:
Test cost: 1.23
Test accuracy: 0.509
Test UAR: 0.5294
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 26   1   0   2]
 [ 17  65   5  23]
 [ 26   3   5  37]
 [ 53  35   8 122]]
Fairness Scores (in terms of equal opportunities):
["ang","sad","hap","neu"]
[0.91836735 0.76415094 0.         0.66745606]
*****************************************************************
Testing model2


...

*****************************************************************
Evaluation on Test Set:
Test cost: 1.34
Test accuracy: 0.383
Test UAR: 0.5373
Test Confusion Matrix:["ang","sad","hap","neu"]
[[25  1  2  1]
 [15 62 21 12]
 [17  6 39  9]
 [39 66 75 38]]
Fairness Scores (in terms of equal opportunities):
["ang","sad","hap","neu"]
[0.84183673 0.67924528 0.86538462 0.87120181]
*****************************************************************
Testing model11


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1320: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


*****************************************************************
Evaluation on Test Set:
Test cost: 1.31
Test accuracy: 0.418
Test UAR: 0.5274
Test Confusion Matrix:["ang","sad","hap","neu"]
[[24  1  3  1]
 [15 72 17  6]
 [14 10 26 21]
 [37 74 50 57]]
Fairness Scores (in terms of equal opportunities):
["ang","sad","hap","neu"]
[0.90659341 0.60787531 0.92444444 0.62857887]
*****************************************************************
Testing model12


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1320: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


*****************************************************************
Evaluation on Test Set:
Test cost: 1.29
Test accuracy: 0.442
Test UAR: 0.5214
Test Confusion Matrix:["ang","sad","hap","neu"]
[[26  1  1  1]
 [21 60  9 20]
 [22  6 18 25]
 [45 55 33 85]]
Fairness Scores (in terms of equal opportunities):
["ang","sad","hap","neu"]
[0.91836735 0.66853646 0.66568047 0.86584875]
*****************************************************************


In [15]:
category_pred = np.argmax(y_pred_enmodel, 1)

In [19]:
enmodel_vote = np.zeros(shape=428)
for s in range(category_pred.shape[0]):
    vote = np.bincount(category_pred[s,:]).argmax()
    enmodel_vote[s] = vote

In [23]:
# compute performance evaluation scores
test_accuracy = accuracy_score(test_dataset.label, enmodel_vote)
test_acc_uw = recall(test_dataset.label, enmodel_vote, average='macro')
test_conf = confusion(test_dataset.label, enmodel_vote)

# compute fairness evaluation scores
fairness_eval = FairnessEvaluation(test_dataset.gender, test_dataset.label, enmodel_vote)

# print results
print("*****************************************************************")
print(f"Evaluation on Test Set:")
print("Test accuracy: %2.3g" %test_accuracy)
print("Test UAR: %3.4g" %test_acc_uw)
print('Test Confusion Matrix:["ang","sad","hap","neu"]')
print(test_conf)
print('Fairness Scores (in terms of equal opportunities):')
fairness_eval.print_equal_opportunities()
print("*****************************************************************")


*****************************************************************
Evaluation on Test Set:
Test accuracy: 0.458
Test UAR: 0.5472
Test Confusion Matrix:["ang","sad","hap","neu"]
[[27  1  1  0]
 [18 65 13 14]
 [24  5 20 22]
 [45 50 39 84]]
Fairness Scores (in terms of equal opportunities):
["ang","sad","hap","neu"]
[0.85714286 0.62954441 0.74175824 0.97835498]
*****************************************************************
